# BERT
This notebook aims to use transfer learning on a BERT model to perform text classification and detect suicidal text.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install -qqq transformers datasets wandb

     |████████████████████████████████| 5.5 MB 5.0 MB/s 
     |████████████████████████████████| 451 kB 54.3 MB/s 
     |████████████████████████████████| 1.9 MB 68.8 MB/s 
     |████████████████████████████████| 182 kB 82.4 MB/s 
     |████████████████████████████████| 7.6 MB 68.7 MB/s 
     |████████████████████████████████| 212 kB 80.3 MB/s 
     |████████████████████████████████| 115 kB 81.4 MB/s 
     |████████████████████████████████| 127 kB 85.7 MB/s 
     |████████████████████████████████| 182 kB 79.5 MB/s 
     |████████████████████████████████| 168 kB 71.2 MB/s 
     |████████████████████████████████| 62 kB 1.5 MB/s 
     |████████████████████████████████| 168 kB 77.2 MB/s 
     |████████████████████████████████| 166 kB 88.1 MB/s 
     |████████████████████████████████| 166 kB 85.4 MB/s 
     |████████████████████████████████| 162 kB 67.9 MB/s 
     |████████████████████████████████| 162 kB 83.4 MB/s 
     |████████████████████████████████| 158 kB 78.5 MB/s 
     |███████████

In [ ]:
# Import packages
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import wandb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset, DatasetDict, load_metric

In [ ]:
# Specify GPU
device = torch.device("cuda")

In [ ]:
# Change to your own directory
try:
    os.chdir("/content/drive/MyDrive/Suicide Project")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Directory changed


## Define constants

In [ ]:
# Define constants
EPOCHS = 1
BATCH_SIZE = 6
LEARNING_RATE = 1e-5
SEED = 4222

MODEL_SAVE_PATH = "Models/bert"
MODEL_CHECKPOINT_PATH = "Models/bert_checkpoint"
MODEL_LOGGING_PATH = "Models/bert_checkpoint/logs"

WANDB_ENTITY = "manit"
WANDB_PROJECT = "Suicide Prediction"
WANDB_RUN = "bert"

## Load dataset

In [ ]:
# Load dataset
df = pd.read_csv('Data/suicide_detection_final_cleaned.csv', header=0, names=['text', 'label', 'cleaned_text'])
df.drop(columns=['cleaned_text'], inplace=True)
df['label'] = df['label'].map({'suicide': 1, 'non-suicide': 0})
df.head()

,text,label
0,Ex Wife Threatening SuicideRecently I left my ...,1
1,Am I weird I don't get affected by compliments...,0
2,Finally 2020 is almost over... So I can never ...,0
3,i need helpjust help me im crying so hard,1
4,It ends tonight.I can’t do it anymore. \nI quit.,1


In [ ]:
# Split dataset into train, validation and test sets
train, temp = train_test_split(df,
                               random_state=SEED,
                               test_size=0.2,
                               stratify=df['label'])

val, test = train_test_split(temp,
                             random_state=SEED,
                             test_size=0.5,
                             stratify=temp['label'])

## Load BERT Model

In [ ]:
# Load BERT tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def dataset_conversion(train, test, val):
  """Converts pandas dataframe to Dataset."""

  train.reset_index(drop=True, inplace=True)
  test.reset_index(drop=True, inplace=True)
  val.reset_index(drop=True, inplace=True)

  train_dataset = Dataset.from_pandas(train)
  test_dataset = Dataset.from_pandas(test)
  val_dataset = Dataset.from_pandas(val)

  return DatasetDict({"train": train_dataset,
                      "test": test_dataset,
                      "val": val_dataset})

raw_datasets = dataset_conversion(train, test, val)

In [ ]:
def tokenize_function(dataset):
    return tokenizer(dataset["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

  0%|          | 0/141 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

  0%|          | 0/18 [00:00<?, ?ba/s]

In [ ]:
# Tokenise datasets
SAMPLE_SIZE = 20
small_train_dataset = tokenized_datasets["train"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
small_test_dataset = tokenized_datasets["test"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))
small_val_dataset = tokenized_datasets["val"].shuffle(seed=SEED).select(range(SAMPLE_SIZE))

full_train_dataset = tokenized_datasets["train"]
full_test_dataset = tokenized_datasets["test"]
full_val_dataset = tokenized_datasets["val"]

In [ ]:
# Import BERT-base pretrained model
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
# Login wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
# Initialise wandb
wandb.init(project=WANDB_PROJECT, entity=WANDB_ENTITY, name=WANDB_RUN)

wandb: Currently logged in as: manit. Use `wandb login --relogin` to force relogin


In [ ]:
# Define custom metrics for computation
def compute_metrics(eval_pred):
    metric_acc = load_metric("accuracy")
    metric_rec = load_metric("recall")
    metric_pre = load_metric("precision")
    metric_f1 = load_metric("f1")
    
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = metric_acc.compute(predictions=predictions, references=labels)["accuracy"]
    recall = metric_rec.compute(predictions=predictions, references=labels)["recall"]
    precision = metric_pre.compute(predictions=predictions, references=labels)["precision"]
    f1 = metric_f1.compute(predictions=predictions, references=labels)["f1"]

    return {"accuracy": accuracy, "recall": recall, "precision": precision, "f1": f1}

In [ ]:
# Define model and training parameters
training_args = TrainingArguments(
    output_dir=MODEL_CHECKPOINT_PATH,
    overwrite_output_dir = True,
    report_to = 'wandb',
    learning_rate=LEARNING_RATE,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    seed=SEED,
    # evaluation_strategy="epoch",
    run_name=WANDB_RUN,
    logging_dir=MODEL_LOGGING_PATH,
    save_strategy="steps",
    save_steps=1500
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

## Pre-trained BERT

In [ ]:
# Predict before fine-tuning
trainer.predict(full_test_dataset).metrics

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 17583
  Batch size = 6
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


{'test_loss': 0.7346116900444031,
 'test_accuracy': 0.4635158960359438,
 'test_recall': 0.9608129843544377,
 'test_precision': 0.4175775292323335,
 'test_f1': 0.5821483942414174,
 'test_runtime': 156.9903,
 'test_samples_per_second': 112.001,
 'test_steps_per_second': 18.67}

## Fine-tuned BERT

In [ ]:
#%%wandb # To observe training progress live

# Fine-tune model
trainer.train()



The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 140660
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 23444
  Number of trainable parameters = 109483778
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss
500,0.279800
1000,0.214700
1500,0.165900
2000,0.196900
2500,0.177200
3000,0.149800
3500,0.141100
4000,0.148700
4500,0.138900
5000,0.151200


Saving model checkpoint to Models/bert_checkpoint/checkpoint-1500
Configuration saved in Models/bert_checkpoint/checkpoint-1500/config.json
Model weights saved in Models/bert_checkpoint/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in Models/bert_checkpoint/checkpoint-1500/tokenizer_config.json
Special tokens file saved in Models/bert_checkpoint/checkpoint-1500/special_tokens_map.json
Saving model checkpoint to Models/bert_checkpoint/checkpoint-3000
Configuration saved in Models/bert_checkpoint/checkpoint-3000/config.json
Model weights saved in Models/bert_checkpoint/checkpoint-3000/pytorch_model.bin
tokenizer config file saved in Models/bert_checkpoint/checkpoint-3000/tokenizer_config.json
Special tokens file saved in Models/bert_checkpoint/checkpoint-3000/special_tokens_map.json
Saving model checkpoint to Models/bert_checkpoint/checkpoint-4500
Configuration saved in Models/bert_checkpoint/checkpoint-4500/config.json
Model weights saved in Models/bert_checkpoint/checkp

TrainOutput(global_step=23444, training_loss=0.12645113970553626, metrics={'train_runtime': 4095.0854, 'train_samples_per_second': 34.348, 'train_steps_per_second': 5.725, 'total_flos': 3.70092010469376e+16, 'train_loss': 0.12645113970553626, 'epoch': 1.0})

In [ ]:
# Resume fine-tuning from checkpoint
trainer.train(MODEL_CHECKPOINT_PATH + "/" + "checkpoint-10500")

Loading model from Models/bert_checkpoint/checkpoint-10500.
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 140660
  Num Epochs = 1
  Instantaneous batch size per device = 6
  Total train batch size (w. parallel, distributed & accumulation) = 6
  Gradient Accumulation steps = 1
  Total optimization steps = 23444
  Number of trainable parameters = 109483778
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0
  Continuing training from global step 10500
  Will skip the first 0 epochs then the first 10500 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your 

  0%|          | 0/10500 [00:00<?, ?it/s]

Step,Training Loss
11000,0.089400
11500,0.097000
12000,0.090900
12500,0.087400
13000,0.094300
13500,0.080700
14000,0.088600
14500,0.075300
15000,0.099100
15500,0.105100


Saving model checkpoint to Models/bert_checkpoint/checkpoint-12000
Configuration saved in Models/bert_checkpoint/checkpoint-12000/config.json
Model weights saved in Models/bert_checkpoint/checkpoint-12000/pytorch_model.bin
tokenizer config file saved in Models/bert_checkpoint/checkpoint-12000/tokenizer_config.json
Special tokens file saved in Models/bert_checkpoint/checkpoint-12000/special_tokens_map.json
Saving model checkpoint to Models/bert_checkpoint/checkpoint-13500
Configuration saved in Models/bert_checkpoint/checkpoint-13500/config.json
Model weights saved in Models/bert_checkpoint/checkpoint-13500/pytorch_model.bin
tokenizer config file saved in Models/bert_checkpoint/checkpoint-13500/tokenizer_config.json
Special tokens file saved in Models/bert_checkpoint/checkpoint-13500/special_tokens_map.json
Saving model checkpoint to Models/bert_checkpoint/checkpoint-15000
Configuration saved in Models/bert_checkpoint/checkpoint-15000/config.json
Model weights saved in Models/bert_check

TrainOutput(global_step=23444, training_loss=0.04918691335410271, metrics={'train_runtime': 2320.2596, 'train_samples_per_second': 60.623, 'train_steps_per_second': 10.104, 'total_flos': 3.70092010469376e+16, 'train_loss': 0.04918691335410271, 'epoch': 1.0})

In [ ]:
# Terminate wandb run
wandb.finish()

eval/accuracy,▁
eval/f1,▁
eval/loss,▁
eval/precision,▁
eval/recall,▁
eval/runtime,▁
eval/samples_per_second,▁
eval/steps_per_second,▁
train/epoch,▁▁▂▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇███▄▅▅▅▆▆▆▆▇▇▇███
train/global_step,▁▁▁▂▂▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇███▄▅▅▅▆▆▆▆▇▇▇███
train/learning_rate,███▇▇▇▆▆▆▆▅▅▅▄▄▄▄▃▃▃▃▂▂▂▁▁▅▄▄▄▄▃▃▃▂▂▂▂▁▁


In [ ]:
# Save fine-tuned model
trainer.save_model(MODEL_SAVE_PATH)

Saving model checkpoint to Models/bert
Configuration saved in Models/bert/config.json
Model weights saved in Models/bert/pytorch_model.bin
tokenizer config file saved in Models/bert/tokenizer_config.json
Special tokens file saved in Models/bert/special_tokens_map.json


In [ ]:
# Evaluate fine-tuned model
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 17583
  Batch size = 6


{'eval_loss': 0.10537157952785492,
 'eval_accuracy': 0.9775351191491782,
 'eval_recall': 0.9698786372276649,
 'eval_precision': 0.9722955145118733,
 'eval_f1': 0.9710855720664666,
 'eval_runtime': 153.5553,
 'eval_samples_per_second': 114.506,
 'eval_steps_per_second': 19.088,
 'epoch': 1.0}

In [ ]:
# Predict after fine-tuning
trainer.predict(full_test_dataset).metrics

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 17583
  Batch size = 6


{'test_loss': 0.10957888513803482,
 'test_accuracy': 0.977307626684866,
 'test_recall': 0.972656821172686,
 'test_precision': 0.9691142191142191,
 'test_f1': 0.9708822885499526,
 'test_runtime': 158.6774,
 'test_samples_per_second': 110.81,
 'test_steps_per_second': 18.471}

In [ ]:
def get_training_history(wandb_run):
  """Extract key metrics from training and eval from wandb run data."""

  # Get training history from wandb
  api = wandb.Api()
  run = api.run(wandb_run)
  history = run.history()

  # Rename columns
  train_column_dict = {'train/epoch': 'epoch', 'train/loss': 'training_loss'}
  val_column_dict = {'train/epoch': 'epoch', 'eval/loss': 'validation_loss', 'eval/accuracy': 'accuracy',
                'eval/precision': 'precision', 'eval/recall': 'recall', 'eval/f1': 'f1'}

  # Train data
  train_history = history[list(train_column_dict.keys())]
  train_history.columns = [train_column_dict.get(x, x) for x in train_history.columns]
  train_history = train_history.dropna()

  # Val data
  val_history = history[list(val_column_dict.keys())]
  val_history.columns = [val_column_dict.get(x, x) for x in val_history.columns]
  val_history = val_history.dropna()

  return pd.merge(train_history, val_history, how="right", on="epoch")

# Get dataframe for training history
WANDB_RUN_ID = "1d0u344o" # Replace with your wandb run details, found in the training cell

training_history = get_training_history(WANDB_ENTITY + "/" + WANDB_PROJECT + "/" + WANDB_RUN_ID)
training_history

,epoch,training_loss,validation_loss,accuracy,precision,recall,f1
0,1.0,NaN,0.105372,0.977535,0.972296,0.969879,0.971086


In [ ]:
# Load fine-tuned model
saved_model = AutoModelForSequenceClassification.from_pretrained(MODEL_SAVE_PATH)

# Load trainer after fine-tune
saved_trainer = Trainer(
    model=saved_model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Predict after fine-tuning
saved_trainer.predict(full_test_dataset).metrics

loading configuration file Models/bert/config.json
Model config BertConfig {
  "_name_or_path": "Models/bert",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.24.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file Models/bert/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initializ

{'test_loss': 0.10957888513803482,
 'test_accuracy': 0.977307626684866,
 'test_recall': 0.972656821172686,
 'test_precision': 0.9691142191142191,
 'test_f1': 0.9708822885499526,
 'test_runtime': 153.871,
 'test_samples_per_second': 114.271,
 'test_steps_per_second': 19.048}

## GPU Memory Utilities

In [ ]:
# Delete variables and empty cache
del trainer
del model
torch.cuda.empty_cache()

In [ ]:
# Python garbage collection
import gc
gc.collect()

43829

In [ ]:
# Check memory allocation
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())

439076352
2443182080


In [ ]:
# Check memory summary
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  428785 KB |    5529 MB |  685459 GB |  685458 GB |
|       from large pool |  428288 KB |    5527 MB |  685363 GB |  685362 GB |
|       from small pool |     497 KB |      20 MB |      95 GB |      95 GB |
|---------------------------------------------------------------------------|
| Active memory         |  428785 KB |    5529 MB |  685459 GB |  685458 GB |
|       from large pool |  428288 KB |    5527 MB |  685363 GB |

In [ ]:
# Check GPU allocation and acprocesses
!nvidia-smi

Sat Nov 26 04:59:02 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   27C    P0    45W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------